1. 문서의 내용을 읽는다.
    %pip install --updrade --quiet  docx2txt langchain_community
2. 문서를 쪼갠다.
    %pip install -qU langchain-text-splitters
    * why ?
    - 토큰 수 초과로 답변을 생성하지 못 할 수 있고
    - 문서가(INPUT)가 길면 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 떄, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [13]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 하나의 청크가 가질 수 있는 토큰의 수
    chunk_overlap=200)  # 청크 간의 겹치는 토큰의 수

loader = Docx2txtLoader("./tax.docx")

#document = loader.load()
document_list = loader.load_and_split(text_splitter=text_splitter)



In [14]:
from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

load_dotenv()
#embedding = OpenAIEmbeddings(model="text-embedding-3-large")

embedding = UpstageEmbeddings(
    model="solar-embedding-1-large" 
)


In [15]:
%pip install -qU langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_chroma import Chroma

#database = Chroma.from_documents(
#    documents=document_list,        # 앞에서 쪼갠 문서를
#    embedding=embedding,             # 임베딩 모델 적용
#    collection_name="chroma-tax",
#    persist_directory=".chroma"
#)
database = Chroma(collection_name="chroma-tax", persist_directory=".chroma",embedding_function=embedding)


In [28]:
query = "연봉 5천만원 직장인의 소득세는 얼마인가요?"
#retrieverd_docs = database.similarity_search(query,k=3) # 유사도 검색


In [19]:
retrieverd_docs

[Document(id='6865bcd6-e331-49df-b9fb-4552d6619152', metadata={'source': './tax.docx'}, page_content='1. 1명인 경우: 연 25만원\n\n2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정

In [29]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-1-mini-chat")


In [30]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자 질문에 답변헤ㅐ 주세요

[Context]
{retrieverd_docs}

Question: {query}
"""

In [31]:
ai_message = llm.invoke(prompt)

In [23]:
ai_message.content

'연봉 5천만원의 직장인의 경우, 세율 및 공제금액 등에 따라 소득세가 달라질 수 있습니다. 그러나 구체적인 공제내역이 없으므로 정확한 소득세를 계산하기는 어렵습니다. 다만, 일반적인 공제 항목을 고려하여 대략적인 소득세를 계산해보겠습니다.\n\n1. **기본 공제**: 배우자, 부양가족 등이 있을 경우 추가 공제가 적용될 수 있습니다. 예를 들어, 1인 가구라면 기본 공제만 적용됩니다.\n   \n2. **자녀세액공제**: 자녀가 있는 경우 자녀세액공제가 적용됩니다. 예를 들어, 1명의 자녀가 있다면 55만원의 공제가 가능합니다.\n\n3. **연금계좌세액공제**: 연금저축계좌에 납입한 금액이 있을 경우 공제가 가능합니다. 예를 들어, 400만원을 납입했다면 최대 48만원(400만원 * 12%)의 공제가 가능합니다.\n\n4. **기타 공제**: 의료비, 교육비, 기부금 등 다양한 공제 항목이 있습니다.\n\n이러한 공제 항목을 고려하여 대략적인 소득세를 계산해보면, 대략 150만원에서 200만원 사이의 소득세가 발생할 수 있습니다. 그러나 이는 일반적인 경우이며, 구체적인 공제 내역에 따라 달라질 수 있습니다. 정확한 소득세를 계산하려면 개인의 공제 내역을 상세히 파악해야 합니다.'

'연봉 5천만원의 직장인의 소득세를 계산하기 위해서는 다음과 같은 과정을 거칩니다.\n\n1. 근로소득공제: 5천만원 x 15% = 750만원\n2. 근로소득금액: 5천만원 - 750만원 = 4천2백5십만원\n3. 기본공제: 150만원\n4. 표준세액공제: 13만원\n5. 근로소득세액공제: 66만원\n\n세액공제 전 세액 = (4천2백5십만원 - 150만원) x 15% - 13만원 - 66만원 = 4천2백8십7만5천원\n\n세액공제 후 세액 = 4천2백8십7만5천원 - 66만원 = 4천2백2십1만5천원\n\n따라서, 연봉 5천만원의 직장인의 소득세는 약 4천2백2십1만5천원입니다. 그러나 이는 기본적인 계산이며, 실제 세금 계산에는 다양한 공제 및 추가 세금이 포함될 수 있습니다. 따라서 정확한 세금 계산을 위해서는 국세청 홈택스 등을 통해 자세한 정보를 확인하시는 것이 좋습니다.'

In [3]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain_classic import hub

#prompt = hub.pull("langchain-ai/solar-1-mini-chat")
#llm = hub.pull("langchain-ai/solar-1-mini-chat")

prompt = hub.pull("rlm/rag-prompt")

In [25]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [36]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)


ValidationError: 1 validation error for LLMChain
prompt
  Input should be a valid dictionary or instance of BasePromptTemplate [type=model_type, input_value="[Identity]\n- 당신은 ...는 얼마인가요?\n", input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type

In [37]:
ai_message = qa_chain.invoke({"query": query})

NameError: name 'qa_chain' is not defined